In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import os

In [19]:
def polynomial(x):
    y1 = x[:, 0] * x[:, 1] * x[:, 2] + 1.2 * x[:, 0] * x[:, 4] - 0.1 * x[:, 5] * x[:, 6] * x[:, 7] - 2 * x[:, 0]**2 * x[:, 7] + x[:, 4]
    y2 = x[:, 0] * x[:, 4] * x[:, 5] - x[:, 2] * x[:, 3] - 3 * x[:, 1] * x[:, 2] + 2 * x[:, 1]**2 * x[:, 3] - 2 * x[:, 6] * x[:, 7] - 1
    y3 = x[:, 2]**2 - x[:, 4] * x[:, 6] - 3 * x[:, 0] * x[:, 3] * x[:, 5] - x[:, 0]**2 * x[:, 2] * x[:, 4] - 1
    y4 = -x[:, 5]**3 + 2 * x[:, 0] * x[:, 2] * x[:, 7] - x[:, 0] * x[:, 3] * x[:, 6] - 2 * x[:, 4]**2 * x[:, 1] * x[:, 3] - x[:, 7]
    y5 = x[:, 0]**2 * x[:, 4] - 3 * x[:, 2] * x[:, 3] * x[:, 7] + x[:, 0] * x[:, 1] * x[:, 3] - 3 * x[:, 5] + x[:, 0]**2 * x[:, 6] + 2
    y6 = x[:, 0]**2 * x[:, 2] * x[:, 5] - x[:, 2] * x[:, 4] * x[:, 6] + x[:, 2] * x[:, 3] + 2.2 * x[:, 1] * x[:, 2]**2 - 1.1
    return np.stack([y1, y2, y3, y4, y5, y6], axis=-1)

In [20]:
def mse_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true))

In [21]:
np.random.seed(0)
Nt = 1000
Nv = 100
noise_sigma = 0.001

print(tf.config.list_physical_devices('GPU'))

# Generate the input data
x_train = np.random.uniform(low=-1, high=1, size=(Nt, 8))
x_val = np.random.uniform(low=-1, high=1, size=(Nv, 8))

# Generate the output data with noise
y_train = polynomial(x_train) + np.random.normal(scale=noise_sigma)
y_val = polynomial(x_val)

[]


In [22]:
def build_model(hidden_layer_nodes, activation_fn):
    model = models.Sequential()
    model.add(layers.Dense(hidden_layer_nodes, activation=activation_fn))
    model.add(layers.Dense(hidden_layer_nodes, activation=activation_fn))
    model.add(layers.Dense(hidden_layer_nodes, activation=activation_fn))
    model.add(layers.Dense(6))
    return model

def train_model(model, x_train, y_train, x_val, y_val, learning_rate, epochs):
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    model.compile(optimizer=optimizer, loss=mse_loss, metrics=['mae'])
    history = model.fit(x_train, y_train, batch_size=32, epochs=epochs, validation_data=(x_val, y_val))
    return history

# Train the model with various configurations
activations = ['relu', 'tanh', 'sigmoid']
learning_rates = [0.01, 0.001, 0.0001]
epochs = [1000, 1500, 2000]

for activation in activations:
    for learning_rate in learning_rates:
        for epoch in epochs:
            model = build_model(6, activation)
            history = train_model(model, x_train, y_train, x_val, y_val, learning_rate, epoch)
            print(f'Training and validation errors: {history.history["loss"][-1]:.4f}, {history.history["val_loss"][-1]:.4f}')

In [23]:
def add_nodes_with_training(model, x_train, y_train, x_val, y_val, learning_rate, epochs, nodes_to_add, hidden_layer_nodes):
    for i in range(3):
        for j in range(nodes_to_add):
            model.add(layers.Dense(hidden_layer_nodes + 1, activation=activation))
            history = train_model(model, x_train, y_train, x_val, y_val, learning_rate, epochs)
            print(f'Training and validation errors with {hidden_layer_nodes + 1} nodes in layer {i + 1}: {history.history["loss"][-1]:.4f}, {history.history["val_loss"][-1]:.4f}')

In [ ]:
Nt = 1100
Nv = 110
x_train = np.random.uniform(low=-1, high=1, size=(Nt, 8))
y_train = polynomial(x_train) + np.random.normal(scale=noise_sigma, size=y_train.shape)
x_val = np.random.uniform(low=-1, high=1, size=(Nv, 8))
y_val = polynomial(x_val)

# add_nodes_with_training(model, x_train, y_train, x_val, y_val, learning_rate, epochs, 2)